In [ ]:
import os
import openai
import pandas as pd
import json

In [ ]:
os.environ["OPENAI_API_KEY"]=""

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
styles=pd.read_csv('architect_styles.csv')
styles.head()

In [ ]:
try:
    with open("architect_styles.json", 'tr') as fi:
        architects_by_style=json.load(fi)
except:
    architects_by_style={}

In [ ]:
architects_by_style.keys()

In [ ]:
completed=False
while not completed:
    completed=True
    for style in styles['style']:
        if style not in architects_by_style or 'architects' not in architects_by_style[style]:
            try:
                print(f"Query {style}")
                completion = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[ {"role": "user", "content": f'List 3 famous architects with name, birth date, death date, nationality, description, and notable buildings for "{style}" architecture style as JSON'}]
                )
                reply = completion.choices[0].message.content
                try:
                    architects_by_style[style]={'architects':json.loads(reply)}['architects']
                except:
                    print(f"Error decoding {style}")
                    print(reply)
                    architects_by_style[style]={'reply':reply}
                with open("architect_styles.json", 'tw') as fo:
                    json.dump(architects_by_style, fo, indent=4)
                completed=False
            except:
                print(f"Error processing {style}")
        else:
            print(f"Skip {style}")

In [ ]:
with open("architect_styles.json", 'tw') as fo:
    json.dump(architects_by_style, fo)

In [ ]:
completed=False
while not completed:
    completed=True
    for style in styles['style']:
        if style not in architects_by_style or 'style' not in architects_by_style[style] or 'period' not in architects_by_style[style]['style']:
            try:
                print(f"Query {style}")
                completion = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": f'What is the time range, period, description, characteristics, examples, continent, and country of the "{style}" architecture style as JSON'}]
                )
                reply = completion.choices[0].message.content
                try:
                    if style not in architects_by_style: architects_by_style[style]={}
                    architects_by_style[style]['style']=json.loads(reply)
                except:
                    print(f"Error decoding {style}")
                    print(reply)
                    architects_by_style[style]['reply2']=reply
                with open("architect_styles.json", 'tw') as fo:
                    json.dump(architects_by_style, fo, indent=4)
                completed=False
            except:
                print(f"Error processing {style}")
        else:
            print(f"Skip {style}")

In [ ]:
completed=False
while not completed:
    completed=True
    for style in styles['style']:
        if style not in architects_by_style or 'terms' not in architects_by_style[style]:
            try:
                print(f"Query {style}")
                completion = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": f'summarize "{style}" architecture style with nouns and adjectives separated by ,'}]
                )
                reply = completion.choices[0].message.content
                try:
                    if style not in architects_by_style: architects_by_style[style]={}
                    architects_by_style[style]['terms']=reply
                except:
                    print(f"Error decoding {style}")
                    print(reply)
                with open("architect_styles.json", 'tw') as fo:
                    json.dump(architects_by_style, fo, indent=4)
                completed=False
            except:
                print(f"Error processing {style}")
        else:
            print(f"Skip {style}")

In [ ]:
for k, v in architects_by_style.items():
    if 'architects' not in v: print("MISSING architects", k)
    for a in v['architects']:
        if 'name' not in a: print("MISSING architect name", k, a)
    if 'terms' not in v: print("MISSING terms", k)
    else:
        if ':' in v['terms'] or '\n' in v['terms']: v['terms']=v['terms'].replace('Nouns:','').replace('Adjectives:','').replace('\n','')
    if 'style' not in v: print("MISSING style", k)
    if 'time_range' not in v['style']: print("MISSING time_range", k)
    if 'period' not in v['style']: print("MISSING period", k)
    if 'description' not in v['style']: print("MISSING description", k)
    if 'characteristics' not in v['style']: print("MISSING characteristics", k)
    else:
        if isinstance(v['style']['characteristics'],str): 
            print("Characteristics should be a list", k)
            v['style']['characteristics']=v['style']['characteristics'].split(',')
    if 'examples' not in v['style']: print("MISSING examples", k)
    else:
        if isinstance(v['style']['examples'],str) and ',' in v['style']['examples']: 
            print("Examples should be a list", k)
            v['style']['examples']=v['style']['examples'].split(',')
    if 'continent' not in v['style']: print("MISSING continent", k)
    else:
        if isinstance(v['style']['continent'],str) and ',' in v['style']['continent']: 
            print("Continent should be a list", k)
            v['style']['continent']=v['style']['continent'].split(',')
    if 'country' not in v['style']: print("MISSING country", k)
    else:
        if isinstance(v['style']['country'],str) and ',' in v['style']['country']: 
            print("Country should be a list", k)
            v['style']['country']=v['style']['country'].split(',')

In [ ]:
with open("architect_styles.json", 'tw') as fo:
    json.dump(architects_by_style, fo, indent=4)

In [ ]:
completed=False
while not completed:
    completed=True
    for index, row in styles.iterrows():
        style=row['style']
        architects_by_style[style]['Start_Year']=row['Start_Year']
        architects_by_style[style]['End_Year']=row['End_Year']

In [ ]:
#with open("architect_styles_old.json", 'tw') as fo:
#    json.dump(architects_by_style, fo, indent=4)

In [ ]:
for fn in os.listdir('styles120'):
    if fn.endswith(".png"):
        name=fn.replace('.png', '').replace('_', ' ').title()
        if name in architects_by_style:
            architects_by_style[name]["icon"]=fn
        else:
            print("Could not find "+name)

In [ ]:
with open("architect_styles.json", 'tw') as fo:
    json.dump(architects_by_style, fo, indent=4)

In [ ]:
architects_by_style_sub={k:architects_by_style[k] for k in styles['style'] if k in architects_by_style and "icon" in architects_by_style[k]}

In [ ]:
with open("architect_styles_sub.json", 'tw') as fo:
    json.dump(architects_by_style_sub, fo, indent=4)

In [ ]:
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff

In [ ]:
styles['Start_Year']=styles['Start_Year'].apply(lambda y: int(y.replace(' CE','').replace('present','2024')) if 'BCE' not in y else -int(y.replace(' BCE','')))
styles['End_Year']=styles['End_Year'].apply(lambda y: int(y.replace(' CE','').replace('present','2024')) if 'BCE' not in y else -int(y.replace(' BCE','')))
styles['Length']=styles['End_Year']-styles['Start_Year']

styles['startY']=styles['Start_Year']

In [ ]:
styles=styles.sort_values(by=['Start_Year','End_Year'])

In [ ]:
fig = go.FigureWidget()
fig.add_bar(x=styles['style'], y=styles['Length'], base=styles['Start_Year'])#, text=styles['style'], textangle=90, textposition='outside',textfont={'family':"Courier New, monospace","size":18, "color":"RebeccaPurple"})
fig.update_layout( autosize=False, width=800, height=800)
fig.show()

In [ ]:
fig = px.timeline(styles.tail(), x_start="Start_Year", x_end="End_Year", y="style")
#fig.update_yaxes(autorange="reversed")
fig.update_traces(width=0.025)  #This update generates a thinner bar (like a line) instead of  the default horizontal bar
fig.show()